Execute a célula de baixo se ainda não instalou o `Watson Developer Cloud Python SDK`. Essa é a biblioteca que vai te permitir conectar, via Python, aos serviços da IBM Cloud.

In [10]:
!pip install --upgrade ibm-watson

Requirement already up-to-date: ibm-watson in /usr/local/lib/python3.6/dist-packages (4.7.1)


Após a instalação, você deve importar os seguintes serviços suficientes para o desafio:

In [11]:
# Lidar com arquivos JSON
import json

# Aqui importamos as classes que cuidam dos serviços do Natural Language Understanding e do Speech-to-Text
from ibm_watson import NaturalLanguageUnderstandingV1, SpeechToTextV1

# Puxamos, para o natural language understanding, as classes de Features e EntitiesOptions que serão úteis para obter as entidades e os sentimentos associdados
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions

# Só é possível se conectar aos seus serviços se você se autenticar, e a classe que cuidará disso é o IAMAuthenticator
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

## Serviço STT

Aqui você só vai precisar da key e da URL do seu serviço do `Speech-to-Text`

In [12]:
stt_apikey = "RdUOgxnnPUixu0YFvTfSN37LI5GvAPgsbIeBzTzUaSJB"

stt_service_url = "https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/937e0133-fbe0-4a5a-a75b-34e7deaddfeb"

In [13]:
stt_authenticator = IAMAuthenticator(apikey=stt_apikey)

stt_service = SpeechToTextV1(authenticator=stt_authenticator)

stt_service.set_service_url(stt_service_url)

## Serviço NLU

Você precisará de 3 coisas: A key e a URL do seu serviço de `Natural Language Understanding` e o model_id do seu Knowledge Studio treinado.

In [14]:
nlu_apikey = "OrKD4bZXjns5eW-zb4RjXL1c2PZEhuynwPo_xEhhAwdv"
    
nlu_service_url = "https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/45e5fdd6-b607-4093-b883-d1a60b2bba7b"
    
nlu_entity_model = "f1eb0755-cd73-43d3-9d7c-fbde8c83c9af"

Agora instanciamos os serviços com as suas credenciais.

In [15]:
# Cria-se um autenticador
nlu_authenticator = IAMAuthenticator(apikey=nlu_apikey)

# Criamos o serviço passando esse autenticador
nlu_service = NaturalLanguageUnderstandingV1(
    version='2018-03-16',
    authenticator=nlu_authenticator)

# Setamos a URL de acesso do nosso serviço
nlu_service.set_service_url(nlu_service_url)

E agora instanciamos o serviço com as suas credenciais.

## Resposta STT

Como enviamos um arquivo temos que abrí-lo primeiro (quando você estiver montando sua api você vai receber como requisição e não como leitura do disco, isso é apenas para exemplo). Você deve enviar um arquivo de tipo `bytes` para o serviço. O `open` do python cuida dessa parte para esse caso. Não carregue um arquivo que não seja formato `.flac`, pois não irá funcionar.



In [16]:
#caminho_para_audio = "./doc/source/dataset/audio_sample.flac"
caminho_para_audio = ""

Como o serviço de Speech-to-Text lida com linguagem, você deve indicar qual modelo linguístico você quer usar. Queremos utilizar um que identifique o português do Brasil. Eu já o separei abaixo para você.

In [17]:
stt_model = 'pt-BR_BroadbandModel'

Com isso pronto, carregamos nosso arquivo `.flac` e o enviamos ao serviço. Isso é feito agora pelo método `.recognize()`. Feito isso você verá o resultado ser printado logo abaixo.

In [18]:
with open('audio_sample.flac', 'rb') as audio_file:
    stt_results = stt_service.recognize(
            audio=audio_file,
            content_type='audio/flac',
            model=stt_model,
            timestamps=False,
            word_confidence=False).get_result()
    
print(json.dumps(stt_results, indent=2, ensure_ascii=False))

{
  "result_index": 0,
  "results": [
    {
      "final": true,
      "alternatives": [
        {
          "transcript": "esse carro espetacular visual indício com bela cabalmente que mídia compatível com a droga de alto e a colocar entretanto o consumo deixa a desejar faço de sete a nove quilômetros livre na cidade com gasolina ",
          "confidence": 0.86
        }
      ]
    }
  ]
}


Uma vez feito isso, sua variável `stt_results` é agora um `dict` (dicionário) com a resposta do serviço ao arquivo de áudio que você forneceu. Você pode agora manipular como quiser essa resposta em seu código para completar o desafio.

In [19]:
# Defina o que você quer enviar para o seu modelo analisar. O importante é que você envie uma string. Essa variável text pode ser o que você quiser
text = stt_results['results'][0]['alternatives'][0]['transcript']

# O método analyze cuida de tudo
nlu_response = nlu_service.analyze(
    text=text,
    features=Features(entities=EntitiesOptions(model=nlu_entity_model, sentiment=True)),
    language='pt'
).get_result()

print(json.dumps(nlu_response, indent=2, ensure_ascii=False))

{
  "usage": {
    "text_units": 1,
    "text_characters": 209,
    "features": 1
  },
  "language": "pt",
  "entities": []
}


## Resposta NLU

Para usar o serviço agora, basta mandar uma variável contendo seu texto para o serviço utilizando o método `.analyze()`. É no próprio método que, na parte de `Features`, você define o modelo que quer usar para extrair as entidades e o sentimento. Lembre que a variável deve estar do formato `str`, ou seja, deve ser uma string. 

Quando rodar a célula abaixo, você verá o resultado do `nlu_response` logo em seguida. Deve demorar alguns segundos para fazer a requisição.

In [75]:
# Defina o que você quer enviar para o seu modelo analisar. O importante é que você envie uma string. Essa variável text pode ser o que você quiser
text = "O novo Fiat 500 apresenta um motor incrível! ótimo ar-condicionado, Sem dúvida possui um dos melhores arranques que eu conheço. O design exterior do veículo também é de chamar muita atenção!"

# O método analyze cuida de tudo
nlu_response = nlu_service.analyze(
    text=text,
    features=Features(entities=EntitiesOptions(model=nlu_entity_model, sentiment=True)),
    language='pt'
).get_result()

print(json.dumps(nlu_response, indent=2, ensure_ascii=False))

{
  "usage": {
    "text_units": 1,
    "text_characters": 190,
    "features": 1
  },
  "language": "pt",
  "entities": [
    {
      "type": "MODELO",
      "text": "Fiat 500",
      "sentiment": {
        "score": 0,
        "label": "neutral"
      },
      "disambiguation": {
        "subtype": [
          "NONE"
        ]
      },
      "count": 1,
      "confidence": 0.998818
    },
    {
      "type": "DESIGN",
      "text": "design exterior do veículo",
      "sentiment": {
        "score": 0,
        "label": "neutral"
      },
      "disambiguation": {
        "subtype": [
          "NONE"
        ]
      },
      "count": 1,
      "confidence": 0.920775
    },
    {
      "type": "ACESSORIOS",
      "text": "ar-condicionado",
      "sentiment": {
        "score": 0.974326,
        "label": "positive"
      },
      "disambiguation": {
        "subtype": [
          "NONE"
        ]
      },
      "count": 1,
      "confidence": 0.788814
    }
  ]
}


Uma vez feito isso, sua variável `nlu_result` é agora um `dict` (dicionário) com a resposta do serviço ao texto que você forneceu. Você pode agora manipular como quiser essa resposta em seu código para completar o desafio.

## Boa Sorte!!

In [76]:
tamanho = len(nlu_response['entities'])
carros = ['TORO','DUCATO','FIORINO','CRONOS','FIAT 500','MAREA','LINEA','ARGO','RENEGADE']
caracteristica = []
sentimento = []
texto = []
carro = ''

In [81]:
for i in range(tamanho):
  entidade = nlu_response['entities'][i]
  if entidade['type'] == 'MODELO':
    carro = entidade['text'].upper().split()
    if ' '.join(carro) == 'FIAT 500':
      carro = 'FIAT 500'
    else:
      carro = set(carro) & set(carros)
      carro = ''.join(carro)
  else:
    caracteristica.append(entidade['type'])
    sentimento.append(entidade['sentiment']['score'])
    texto.append(entidade['text'])

In [82]:
caracteristicas = []
for a, b, c in zip(caracteristica, sentimento, texto):
  caracteristicas.append({"entity": a,"sentiment": b,"mention": c})

In [83]:
result_true = {"recommendation": carro,"entities": caracteristicas}
result_false = {"recommendation": "","entities": []}

In [84]:
print(json.dumps(result_true, indent=2, ensure_ascii=False)) 

{
  "recommendation": "FIAT 500",
  "entities": [
    {
      "entity": "DESIGN",
      "sentiment": 0,
      "mention": "design exterior do veículo"
    },
    {
      "entity": "ACESSORIOS",
      "sentiment": 0.974326,
      "mention": "ar-condicionado"
    },
    {
      "entity": "DESIGN",
      "sentiment": 0,
      "mention": "design exterior do veículo"
    },
    {
      "entity": "ACESSORIOS",
      "sentiment": 0.974326,
      "mention": "ar-condicionado"
    }
  ]
}


In [41]:
import pandas as pd

label = ['CARAC','TORO','DUCATO','FIORINO','CRONOS','FIAT 500','MAREA','LINEA','ARGO','RENEGADE']
data = [
        ['ACESSORIOS',8,2,1,6,7,4,3,5,9],
        ['CONFORTO',5,2,1,6,9,3,8,4,7],
        ['CONSUMO',3,6,7,4,8,1,2,9,5],
        ['DESEMPENHO',9,4,1,6,2,5,8,3,7],
        ['DESIGN',8,2,1,6,9,3,7,4,5],
        ['MANUTENCAO',3,7,9,4,1,2,6,8,5],
        ['SEGURANCA',7,3,1,5,9,2,4,6,8]
        ]

In [43]:
df = pd.DataFrame(data=data, columns=label)

In [44]:
df.set_index('CARAC', inplace=True)

['DESIGN', 'ACESSORIOS']

In [73]:
df.drop(columns=carro).loc[caracteristica].sum()

DUCATO       4
FIORINO      2
CRONOS      12
FIAT 500    16
MAREA        7
LINEA       10
ARGO         9
RENEGADE    14
dtype: int64

In [74]:
df.drop(columns=carro).loc[caracteristica].sum().idxmax()

'FIAT 500'

In [68]:
carro

'TORO'